# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# jupyter nbextension enable --py gmaps

# Not generating??

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
file = pd.read_csv("../output_file/city.csv")
file.dropna()
file.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,East London,12,ZA,1621375736,34,-33.0153,27.9116,66.07,6.87
1,1,Eureka,90,US,1621375802,70,40.8021,-124.1637,65.68,10.00
2,2,Poum,99,NC,1621375802,72,-20.2333,164.0167,75.36,10.94
3,3,Rikitea,67,PF,1621375803,68,-23.1203,-134.9692,74.57,18.05
4,4,Rome,75,US,1621375597,33,43.2128,-75.4557,78.66,13.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng'
location = file[["Latitude", "Longitude"]]
humidity = file["Humidity"].astype(float)

In [6]:
# Plot Heatmap
figure = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(location, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
figure.add_layer(heat_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
temperature_file=file.loc[(file['Max Temperature']>=70)&(file['Max Temperature']<=80)]
wind_file=temperature_file.loc[(temperature_file['Wind Speed']<=10)]
normal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
normal_weather.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
282,282,Al Bardīyah,0,LY,1621375894,55,31.7561,25.0865,71.15,1.19
344,344,Kiryat Gat,0,IL,1621375622,76,31.6100,34.7642,72.39,3.69
357,357,Saint-Pierre,0,RE,1621375917,55,-21.3393,55.4781,71.28,6.91
530,530,Tūkrah,0,LY,1621375974,47,32.5341,20.5791,72.12,6.17
531,531,Ashquelon,0,IL,1621375974,76,31.6693,34.5715,73.40,5.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = normal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# Add params dictionary
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng to identify hotels in dataframe
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change the location for each iteration
    params["location"] = f"{lat},{lng}"
    
    # Use base_url to make call
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make the requests
    hotel_name = requests.get(base_url, params=params)
    
    # Convert to JSON
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Skip any items that are missing data points
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

{
    "error_message": "You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "You must enable Bil

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
0,282,Al Bardīyah,0,LY,1621375894,55,31.7561,25.0865,71.15,1.19,
1,344,Kiryat Gat,0,IL,1621375622,76,31.6100,34.7642,72.39,3.69,
2,357,Saint-Pierre,0,RE,1621375917,55,-21.3393,55.4781,71.28,6.91,
3,530,Tūkrah,0,LY,1621375974,47,32.5341,20.5791,72.12,6.17,
4,531,Ashquelon,0,IL,1621375974,76,31.6693,34.5715,73.40,5.55,
5,537,Pimentel,0,PE,1621375977,81,-6.8367,-79.9342,71.89,4.00,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

figure.add_layer(markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))